<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Rec_Engine_Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install and Imports

In [2]:
!pip install --upgrade spacy
!python -m spacy download en_core_web_lg
!pip install -U kaleido
!pip install bertopic
# !python -m spacy download en_core_web_lg

     |████████████████████████████████| 6.2 MB 1.1 MB/s 
     |████████████████████████████████| 457 kB 66.3 MB/s 
     |████████████████████████████████| 660 kB 49.3 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 10.1 MB 48.2 MB/s 
     |████████████████████████████████| 181 kB 54.2 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    U

In [3]:
# from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
from google.colab import drive
import os
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

import re
import string

import nltk
nltk.download('stopwords')
import spacy

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import kaleido
nlp = spacy.load('en_core_web_lg')
multiprocessing.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2

## helper functions

In [4]:
import joblib
import os 

def create_path_if_not_exists(datapath):
    '''Create the new file if not exists and save the data'''

    if not os.path.exists(datapath):
        os.makedirs(datapath) 

def remove_punct_df(df):

  # remove punctuation
  # text_col = [''.join(letter for letter in word if letter not in string.punctuation) for word in df]

  filt_combined = []
  for word in word_tokenize(df):

    if word.lower() not in string.punctuation:
      # print(word)
      
      filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip

def create_input_new(df):

  # for title
  indices = df['Title'].isna()
  df.loc[indices,'Title'] = ""

  # for abstract
  indices = df['Abstract'].isna()
  df.loc[indices,'Abstract'] = ""

  # combined - title + abstract
  df['Combined'] = df['Title'] + " " + df['Abstract']

  # remove blanks
  df = df[df['Combined']!=" "]

  #keep only selected cols
  df_sel = df[['Authors','Pub Year','Research Area','Combined',"Pub TYpe"]]
  df_sel= df_sel.rename(columns={'Pub Year':'pub_year',"Research Area":"research_area","Authors":"authors","Combined":"combined","Pub TYpe":"pub_type"})

  combined = list(df_sel['combined'])

  # remove patterns
  pattern = r'<inf>|</inf>|<sup>|</sup>|inf|/inf'
  comb_clean = []
  for l in combined:
    mod_string = re.sub(pattern, '', l )
    comb_clean.append(mod_string)

  # merge back to df
  df_sel['combined'] = comb_clean

  # filter spurtious yeats
  df_sel = df_sel[df_sel['pub_year']!='12.0.1.2']

  # convert years to int
  # df_sel['pub_year'] = df_sel['pub_year'].astype(str)
  df_sel['pub_year'] = df_sel['pub_year'].astype(str).replace('\.0', '', regex=True).astype(int)
  # if year is 201, that is mistyped fom 2001
  df_sel[df_sel['pub_year']==201]['pub_year'] = 2001


  return df_sel

def lemma_spacy(df_combined):

  filt_combined = []
  for word in nlp(df_combined):
    if word.lemma_ != '-PRON-' :
      filt_combined.append(word.lemma_)

  new_df = " ".join(filt_combined)

  return new_df  

def remove_stop_df(df_combined):

  # remove stopwords

  new_df= []
  filt_combined = []
  for word in word_tokenize(df_combined):

    if word.lower() not in stopwords.words('english'):
      # print(word)
      if word.lower() == "perovskites":
        filt_combined.append("perovskite")
      else:
        filt_combined.append(word)
  filtered_ip= " ".join(filt_combined)

  return filtered_ip



def flatten(t):
    return [item for sublist in t for item in sublist]


def get_authors(input_data,rep_docs):
  tt = input_data['combined'].to_list()

  auth_list= []
  for d in rep_docs:
    ind = tt.index(d)
    auth_str = input_data.authors[ind]
    auth_el = auth_str.split(" ,")
    auth_list.append(auth_el)

  auth_list = flatten(auth_list)
  if '' in auth_list:
    auth_list.remove('')

  # unduplicate repeating authors 
  # print("Length of List:",len(auth_list))
  # print(auth_list)


  final_auth_list = list(set(auth_list))
  if '' in final_auth_list:
    final_auth_list.remove('')

  # print(final_auth_list)
  # print("Length of Final List:",len(final_auth_list))

  return final_auth_list

  
def author_all_topics(model,input_data):

  dict_df = {}
  for i in range(len(model.topics)-1):
    rep_docs = model.get_representative_docs(i)
    author_list = get_authors(input_data,rep_docs)

    dict_df[i] = author_list

  # create df with topics and authors
  author_topics = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_df.items() ]))

  return author_topics,dict_df



## path of files

In [5]:
drive.mount('/content/drive')
# read all files
path = '/content/drive/MyDrive/NLP/ALS Spreadsheets/'
files = os.listdir(path)
files.sort()
files
# beamline 2.4 bad ..removed; also one more check which one
# 3.2.1 bad

Mounted at /content/drive


['Beamline_1.4.xls',
 'Beamline_10.0.1.xls',
 'Beamline_10.3.1.xls',
 'Beamline_10.3.2.xls',
 'Beamline_11.0.1.xls',
 'Beamline_11.0.2.xls',
 'Beamline_11.3.1.xls',
 'Beamline_11.3.2.xls',
 'Beamline_12.0.1.1.xls',
 'Beamline_12.0.1.2.xls',
 'Beamline_12.0.1.4.xls',
 'Beamline_12.0.2.xls',
 'Beamline_12.2.1.xls',
 'Beamline_12.2.2.xls',
 'Beamline_12.3.1.xls',
 'Beamline_12.3.2.xls',
 'Beamline_2.1.xls',
 'Beamline_3.1.1.xls',
 'Beamline_3.3.2.xls',
 'Beamline_4.0.2.xls',
 'Beamline_4.0.3.1.xls',
 'Beamline_4.0.3.2.xls',
 'Beamline_4.2.2.xls',
 'Beamline_5.0.1.xls',
 'Beamline_5.0.2.xls',
 'Beamline_5.0.3.xls',
 'Beamline_5.3.1.xls',
 'Beamline_5.3.2.1.xls',
 'Beamline_5.3.2.2.xls',
 'Beamline_5.4.xls',
 'Beamline_6.0.1.xls',
 'Beamline_6.0.2.xls',
 'Beamline_6.1.2.xls',
 'Beamline_6.3.1.xls',
 'Beamline_6.3.2.xls',
 'Beamline_7.0.1.xls',
 'Beamline_7.0.2.xls',
 'Beamline_7.3.1.xls',
 'Beamline_7.3.3.xls',
 'Beamline_8.0.1.xls',
 'Beamline_8.2.1.xls',
 'Beamline_8.2.2.xls',
 'Beamline_

## Rec Engine 

In [5]:
start_time = time.time()
iter_version = "BERTopic_SPECTER_v3/"

for i in range(1):
# for i in range(len(files)):
  print("File:",str(i+1) + ' ' + files[i])
  # beam_name = files[i].split('xls')[0][:-1]
  beam_name = "Beamline_8.3.2.xls"
  

  df = pd.read_table(path + beam_name, on_bad_lines='skip')
  # df = pd.read_table(path + files[i], on_bad_lines='skip')


  # combine abstract and title
  input_data = create_input_new(df)

  # remove stopwords
  input_data['combined'] = input_data['combined'].apply(remove_stop_df)

  # lemmatize
  input_data['combined'] = input_data['combined'].apply(lemma_spacy)

  # remove punct
  input_data['combined'] = input_data['combined'].apply(remove_punct_df)

  # train model
  sentence_model = SentenceTransformer('allenai-specter')
  topic_model_specter = BERTopic(embedding_model=sentence_model,verbose=True)
  list_text = input_data['combined'].to_list()

  topics, probs = topic_model_specter.fit_transform(list_text)
  # print("Before removing:")
  # print(topic_model_specter.topics)

  # topic_model_specter.topics.pop(-1, None)
  # print("After removing")
  # print(topic_model_specter.topics)

  # defining path
  base_path = '/content/drive/MyDrive/NLP/'
  model_path = base_path + iter_version + beam_name + "/"
  create_path_if_not_exists(model_path)

  # save model as pickle file
  file_path = model_path + "model" + beam_name + ".pkl"
  joblib.dump(topic_model_specter, file_path) 


  # dynamic topic modeling
  years = input_data['pub_year'].to_list()
  topics_over_time = topic_model_specter.topics_over_time(list_text, topics, years)
  fig_time = topic_model_specter.visualize_topics_over_time(topics_over_time, top_n_topics=20)
  fig_name_png = model_path + "topic_time" + beam_name  +".png"
  fig_time.write_image(fig_name_png)
  fig_name_html = model_path + "topic_time" + beam_name  +".html"
  fig_time.write_html(fig_name_html)

  # visualize barchart of topics as html
  fig = topic_model_specter.visualize_barchart(top_n_topics = len(topic_model_specter.topics))
  fig_name = model_path + "bar_chart" + beam_name  +".html"
  fig_name_png = model_path + "bar_chart" + beam_name  +".png"

  fig.write_html(fig_name)
  fig.write_image(fig_name_png)


  # save as excel
  # df_info = pd.DataFrame(model.get_topic_info())
  excel_name = model_path + "Topic_Results.xlsx"
  # df_info.to_excel(excel_name,sheet_name="topic_info")

  df_topics = pd.DataFrame(topic_model_specter.topics)
  df_topics.to_excel(excel_name,sheet_name="topic_words")


  ## generating authors for each topic 
  df_authors, dict_authors = author_all_topics(topic_model_specter,input_data)
  # save df with authors 
  excel_name = model_path + "Author_Topics.xlsx"
  df_authors.to_excel(excel_name,sheet_name="authors_topics")

  search_term = input("Find authors and documents related to the term : ")


  similar_topics, similarity = topic_model_specter.find_topics(search_term, top_n=5)
  print("Authors are:",dict_authors[similar_topics[0]])

  # get all docs related to a particular search term 
  rep_docs = topic_model_specter.get_representative_docs(similar_topics[0])
  print("Closest documents are:",rep_docs)

  
  print("File: {} done ".format(i+1) )


print('Total time taken (mins): ', int((time.time()-start_time)/60))



File: 1 Beamline_1.4.xls


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

2022-05-20 16:38:54,693 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-05-20 16:39:05,435 - BERTopic - Reduced dimensionality
2022-05-20 16:39:05,473 - BERTopic - Clustered reduced embeddings
19it [00:00, 32.70it/s]


Find authors and documents related to the term : battery
Authors are: ['Harris, G.', 'Hexemer, Alexander', 'Manuck, C.', 'McElrone, A.', 'Jameson, John R.,Albert, C.', 'Williams, L.', 'Smith, P.', 'Brodersen, C.', 'Patakas, A.', 'Parkinson, Dilworth Y.,Tull, Craig E,', 'Pearsall, K.']
Closest documents are: ['lithium Dendrite Growth Glassy Rubbery Nanostructured Block Copolymer Electrolytes enable use lithium metal anode critical step require dramatically increase energy density rechargeable battery however dendrite growth lithium metal battery lack fundamental understanding factor govern growth limit factor prevent adoption herein present effect battery cycling temperature range 90 120 ° c dendrite growth polystyrene block poly ethylene oxide -based electrolyte temperature range encompass glass transition temperature polystyrene 107 ° c slight increase cycling temperature symmetric lithium polymer lithium cell 90 105 ° c result factor five decrease amount charge pass short circuit syn

### given a doc, find other docs

In [6]:
# 1. given a doc, find the closest topic and then find the most rep docs of that topic
# 2. given a doc, find the closest topic, creat emebedding of each doc in that topic and find the most similar doc to the given doc (?)

search_doc = "Researchers have increased the lifetime of a promising electric vehicle \\
 battery to a record level, an important step toward the goal of lighter, less expensive and long-lasting batteries for future electric vehicles. The work is reported June 28 in the journal Nature Energy.Such batteries—the goal\\
  of research groups the world over—are seen as an important part of the solution to reduce the effects of climate change, and scientists are exploring a dizzying array of options.\\
One solution on the horizon is a lithium-metal battery for electric vehicles. These batteries hold almost twice the energy of their widely used lithium-ion counterparts, and they’re lighter.\\
 That combination offers the enticing prospect of an electric vehicle that would be lighter and go much farther on a single charge. But lithium-metal batteries in the laboratory have been plagued by premature death, lasting only a fraction of the time of today’s lithium-ion batteries."




In [7]:
similar_topics_doc, similarity_doc = topic_model_specter.find_topics(search_doc, top_n=5)


In [8]:
similar_topics_doc

[0, -1, 11, 10, 6]

In [9]:
# simialrity of each word to that topic
topic_model_specter.get_topic(similar_topics_doc[0])

[('lithium', 0.060193396849873536),
 ('electrolyte', 0.05150925431548401),
 ('electrode', 0.04070435171817028),
 ('cell', 0.03929788174574032),
 ('battery', 0.03517805244338436),
 ('density', 0.03097104607655961),
 ('metal', 0.02939032856067009),
 ('catalyst', 0.029126750364824268),
 ('current', 0.02650352793778013),
 ('water', 0.026159475513318986)]

In [11]:
topic_model_specter.get_representative_docs(similar_topics_doc[0])


['lithium Dendrite Growth Glassy Rubbery Nanostructured Block Copolymer Electrolytes enable use lithium metal anode critical step require dramatically increase energy density rechargeable battery however dendrite growth lithium metal battery lack fundamental understanding factor govern growth limit factor prevent adoption herein present effect battery cycling temperature range 90 120 ° c dendrite growth polystyrene block poly ethylene oxide -based electrolyte temperature range encompass glass transition temperature polystyrene 107 ° c slight increase cycling temperature symmetric lithium polymer lithium cell 90 105 ° c result factor five decrease amount charge pass short circuit synchrotron hard x ray microtomography experiment reveal shift dendrite location primarily within lithium electrode 90 ° c primarily within electrolyte 105 ° c rheological measurement show large change mechanical property temperature window time temperature superposition use interpret rheological datum Dendrite

### similarity score of each document

In [ ]:
# give similarity between search doc and the docs of the closest do

In [12]:
!pip install sent2vec

In [13]:
rep_docs

['lithium Dendrite Growth Glassy Rubbery Nanostructured Block Copolymer Electrolytes enable use lithium metal anode critical step require dramatically increase energy density rechargeable battery however dendrite growth lithium metal battery lack fundamental understanding factor govern growth limit factor prevent adoption herein present effect battery cycling temperature range 90 120 ° c dendrite growth polystyrene block poly ethylene oxide -based electrolyte temperature range encompass glass transition temperature polystyrene 107 ° c slight increase cycling temperature symmetric lithium polymer lithium cell 90 105 ° c result factor five decrease amount charge pass short circuit synchrotron hard x ray microtomography experiment reveal shift dendrite location primarily within lithium electrode 90 ° c primarily within electrolyte 105 ° c rheological measurement show large change mechanical property temperature window time temperature superposition use interpret rheological datum Dendrite

In [14]:
from sent2vec.vectorizer import Vectorizer
from scipy import spatial

sentences = [
    search_doc,
    rep_docs[0],
    rep_docs[1],
    rep_docs[2]
]
vectorizer = Vectorizer()
vectorizer.run(sentences)
vectors_bert = vectorizer.vectors

dist_1 = spatial.distance.cosine(vectors_bert[0], vectors_bert[1])
dist_2 = spatial.distance.cosine(vectors_bert[0], vectors_bert[2])
dist_3 = spatial.distance.cosine(vectors_bert[0], vectors_bert[3])
print('dist_1: {0}, dist_2: {1}, dist_3: {2}'.format(dist_1, dist_2,dist_3))

dist_list = [dist_1,dist_2,dist_3]
max_val = max(dist_list)
max_ind = dist_list.index(max_val)

print("Doc closest to the given document is:",rep_docs[max_ind])

Initializing Bert distilbert-base-uncased
Vectorization done on cuda


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


dist_1: 0.16345137357711792, dist_2: 0.15112560987472534, dist_3: 0.32525497674942017
Doc closest to the given document is: understand Water Transport Polymer Electrolyte Fuel Cells use Coupled Continuum Pore Network Models


In [15]:
similar_topics_doc

[0, -1, 11, 10, 6]

In [16]:
# most similar from among all docs
from sent2vec.vectorizer import Vectorizer
from scipy import spatial

input_list = list(input_data['combined'] )
sentences = [search_doc] + input_list

vectorizer = Vectorizer()
vectorizer.run(sentences)
vectors_bert = vectorizer.vectors

# formualate distances with each doc
dist_list=[]
for i in range(len(sentences)-1):
  dist_list.append(spatial.distance.cosine(vectors_bert[0], vectors_bert[i+1]))

max_val = max(dist_list)
max_ind = dist_list.index(max_val)

print("Doc closest to the given document is:",input_list[max_ind])

Initializing Bert distilbert-base-uncased
Vectorization done on cuda


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (569 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: ignored

In [34]:
len(input_list)

510

In [36]:
del(vectorizer)

### find nearest topic to given author and then from the complete list, return all author names 

In [ ]:
# one simple process is to simply search in the author df for the author and return all the other authors in that topic 
# but if author not present in that list (which) is possible since the auth list fromed from the top3 rep docs), then first find all the docs of the given author, find nearest doc to the given doc and return auth of these docs.

In [22]:
# simply search through the topic author df for the given author 
# find the doc in which that author occ, then find closest docs to that doc and return authors for that

# search_author = "Hirano, S."
search_author = "Manga, M."


In [24]:
# iterate thru dict, pick up the topic where that author occurs and return all authors in that topic

search_author = "Manga, M."
for t in dict_authors:
  if search_author in dict_authors[t]:
    print("Topic Number is:",t)
    print("All other authors:",dict_authors[t])



Topic Number is: 1
All other authors: ['Brand, B.D.', 'Brodersen, Craig', 'Rico, Cyren Mendoza,Guenni, O.', 'Voltolini, M.', 'Manga, M.', 'Pittermann, J.', 'Ajo-Franklin, J.B.', 'Marshall, A.A.', 'Huber, C.', 'Valdivia, P.']


In [32]:
# find the doc in which that author occurs, then find closest docs to that doc and return authors for that
relevant_docs = input_data[input_data.apply(lambda row: row.astype(str).str.contains(search_author).any(), axis=1)]['combined'].to_list()


In [33]:
relevant_docs

['externally trigger renew bubble nucleation basaltic magma 12 October 2008 eruption Halemaumau Overlook vent Kilauea Hawaii USA October 2008 present dozen small impulsive explosive eruption occur overlook vent southeast side Halemaumau Crater Klauea volcano USA eruption trigger rockfall wall volcanic vent conduit onto top lava column use microtextural observation datum clast erupt well characterize 12 October 2008 explosive eruption Halemaumau extend exist model eruption trigger present potential mechanism eruption combine microtextural observation exist geophysical visual datum set measure size number density bubble preserve juvenile ejecta use 2D image X ray microtomography datum suggest accumulation large bubble diameter gt 50 least millimeter exist shallow level within conduit prior 12 October 2008 explosion furthermore high number density small bubble lt 50 measured clast imply rapid nucleation bubble visual observation combine preexist geophysical datum suggest impact rockfall o

In [39]:
# find closest author to each doc by using each doc as a search term
# search_term = input("Find authors and documents related to the term : ")


def closest_author_docs(relevant_docs):

  auth_list = []

  for doc in relevant_docs:
    similar_topics, similarity = topic_model_specter.find_topics(doc, top_n=5)
    auth_list.append(dict_authors[similar_topics[0]])
    # print("Authors are:",dict_authors[similar_topics[0]])

  # deduplicate author list 
  # print(auth_list)

  final_auth_list = list(set(flatten(auth_list)))


  return final_auth_list

In [40]:
authors_docs = closest_author_docs(relevant_docs)

In [43]:
authors_docs

['Narter, Matthew E.,Schnarr, G.',
 'Brusseau, Mark',
 'Wu, Ye',
 'Zenyuk, Iryna V,Lamibrac, A.',
 'Ajo-Franklin, J.',
 'Tyler, S.W.',
 'Ibarra, Caitlin Leia,MacDowell, Alastair A.,',
 'Sigwart, Julia',
 'Boran, T.',
 'Taylor, Jeffrey',
 'Kluepfel, Dan',
 'Zhang, Guangyu',
 'Balooch, Guive',
 'Ghezzehei, Teamrat A.,Tyler, S.W.',
 'Berli, Markus',
 'Büchi, F.N.',
 'Knipfer, Thorsten M,Brodersen, Craig',
 'Voltolini, M.',
 'Richards, Gary D.,Jabbour, Rebecca S,Horton, C.F.',
 'Parkinson, Dilworth Y.,Marone, F.',
 'Peterkova, R.',
 'MacDowell, Alastair A.,Weber, Adam Z,',
 'Hubbard, S.',
 'Marble, Justin C.,',
 'Gostick, Jeff T.,Gunterman, Haluna Penelope F.,Kienitz, Brian L.,Newman, James',
 'Marshall, Grayson W.,Basson, M.A.',
 'Smith, Ryan',
 'Peterka, M.',
 'Weber, Adam Z,',
 'Aravena, J.E.',
 'Zedan, Amr',
 'Lyons, D.B.',
 'Fujita, Y.',
 'Eller, J.h.',
 'Klein, O.D.',
 'Williams, K.',
 'McElrone, Andrew Joseph,',
 'Martin, G.R.',
 'Harry, Katherine',
 'Spycher, N.']

In [44]:
 other_list= ['Brand, B.D.', 'Brodersen, Craig', 'Rico, Cyren Mendoza,Guenni, O.', 'Voltolini, M.', 'Manga, M.', 'Pittermann, J.', 'Ajo-Franklin, J.B.', 'Marshall, A.A.', 'Huber, C.', 'Valdivia, P.']

In [45]:
list(set(authors_docs) & set(other_list))

['Voltolini, M.']

In [1]:
final_auth_list = authors_docs + other_list
final_auth_list = list(set(final_auth_list))
final_auth_list

NameError: ignored